In [1]:
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
from VolvoDataset import VolvoDataset
import pandas as pd
import numpy as np
import torch
import os

In [2]:
#On Jack pc
DATA_DIR = r"C:\Users\giaco\OneDrive\Desktop\TimeSeries\Volvo\train_gen1.csv"
#On Heron 
#TRAIN
DATA_DIR = r"/data1/malto/volvo_ecml_2024/train_gen1.csv"
#TEST
TEST_DATA_DIR = r"/data1/malto/volvo_ecml_2024/public_X_test.csv"


In [3]:
train_dataset = VolvoDataset(DATA_DIR)

--- Train Dataset ---
preprocessing ... 
Dropping all NaN column
Dropping all static columns
Grouping by Chassis id


In [4]:
test_dataset = VolvoDataset(TEST_DATA_DIR, test=True, columns_to_keep=train_dataset.kept_columns)

--- Test Dataset ---
Grouping by Chassis id


In [5]:
test_dataset.volvo_df.head()

,Timesteps,ChassisId_encoded,gen,af1__0,af1__1,af1__2,af1__3,af1__4,af1__5,af1__6,...,f__233,f__234,f__235,f__236,f__237,f__238,f__239,f__240,f__241,f__242
0,0.0,181.0,gen2,0.001572,0.000074,0.009537,2.436562e-07,0.005458,0.005413,0.003982,...,0.228181,2.260611,0.443237,-0.173428,-0.490855,0.027760,-0.007520,0.111756,0.713604,-0.108912
1,1.0,181.0,gen2,0.001572,0.000074,0.009537,2.436562e-07,0.005458,0.005413,0.003982,...,0.228181,2.260611,0.443237,-0.150103,-0.490656,0.016915,-0.012925,0.126104,0.752002,-0.106688
2,2.0,181.0,gen2,0.001572,0.000074,0.009537,2.436562e-07,0.005458,0.005413,0.003982,...,0.228181,2.260611,0.443237,-0.136927,-0.491369,0.014565,-0.015026,0.126458,0.764391,-0.105829
3,3.0,181.0,gen2,0.001572,0.000074,0.009537,2.436562e-07,0.005458,0.005413,0.003982,...,0.228181,2.260611,0.443237,-0.127777,-0.489940,0.016210,-0.015648,0.125425,0.760781,-0.105922
4,4.0,181.0,gen2,0.001572,0.000074,0.009537,2.436562e-07,0.005458,0.005413,0.003982,...,0.228181,2.260611,0.443237,-0.112876,-0.491121,0.014534,-0.016847,0.127709,0.782974,-0.101559


In [6]:
test_dataset.get_n_features(), train_dataset.get_n_features()

(289, 289)

In [7]:
train_dataloader = DataLoader( train_dataset, batch_size = 12, collate_fn = VolvoDataset.padding_collate_fn)
test_dataloader = DataLoader( test_dataset, batch_size = 12, collate_fn = VolvoDataset.padding_collate_fn)

In [8]:
train_dataset.df_list[4].shape

(13, 293)

In [9]:
for batch in train_dataloader:
    data, labels, mask = batch
    print("BEFORE")
    print(f"mask: {mask.shape}")
    print(f"data: {data.shape}")
    print(f"labels: {labels.shape}")
    
    
    mask = mask.reshape(-1)
    data = data.reshape(-1)
    labels = labels.reshape(-1,3)
    print("AFTER")    
    print(f"mask: {mask.shape}")
    print(f"data: {data.shape}")
    print(f"labels: {labels.shape}")
    
    labels = labels[ mask.type(torch.bool) ]
    
    res_df = pd.DataFrame(train_dataset.risk_encoder.inverse_transform(labels), columns=["pred"])
    #res_df.to_csv("shit.csv", index=False)
    print(mask.sum())
    break
    

BEFORE
mask: torch.Size([12, 29])
data: torch.Size([12, 29, 289])
labels: torch.Size([12, 29, 3])
AFTER
mask: torch.Size([348])
data: torch.Size([100572])
labels: torch.Size([348, 3])
tensor(245.)


In [10]:
for batch in test_dataloader:
    pass
